260107.16.49
This file create for try to find solution for global feeback 

In [68]:
import re

In [69]:
import yaml
import json
import os

class LiteralString(str):
    """Force YAML to use block literal style '|'."""
    pass
def literal_representer(dumper,value):
    """Represent LiteralString using YAML block style."""
    return dumper.represent_scalar('tag:yaml.org,2002:str', value, style='|')

# Register custom representer
yaml.add_representer(LiteralString, literal_representer)

class Helper:
    """
    Utility helper class for file I/O, YAML/JSON handling,
    formatting, and configuration transformations.
    """
    @staticmethod
    def load_file(filepath: str) -> str:
        """
        Load and return the contents of a text file.
        Args:
            filepath (str): Path to the file.
        Returns:
            str: File contents.
        """
        with open(filepath, "r", encoding="utf-8") as file:
            return file.read()
        
    @staticmethod
    def load_yaml(filepath: str) -> dict:
        """
        Load a YAML file and return its contents as a dictionary.
        Args:
            filepath (str): Path to the YAML file.
        Returns:
            dict: Parsed YAML content.
        """
        with open(filepath, "r", encoding="utf-8") as f:
            return yaml.safe_load(f)
        
    @staticmethod
    def load_json(filepath: str) -> dict:
        """
        Load a JSON file and return its contents as a dictionary.
        Args:
            filepath (str): Path to the JSON file.
        Returns:
            dict: Parsed JSON content.
        """
        with open(filepath, "r", encoding="utf-8") as f:
            return json.load(f)
        
    @staticmethod
    def save_yaml(newconfig,filepath: str):
        """
        Save a dictionary to a YAML file.
        Preserves key order and applies custom representers
        such as block literal formatting.
        Args:
            newconfig (dict): Configuration data to save.
            filepath (str): Output YAML file path.
        """
        with open(filepath,"w",encoding="utf-8") as file:
            return yaml.dump(newconfig,file,sort_keys=False)
        
    @staticmethod
    def fop(num: float) -> float:
        """
        Format a number to one decimal place.
        Args:
            num (float): Input number.
        Returns:
            float: Rounded number with one decimal precision.
        """
        return float(f"{num:.1f}")
    
    @staticmethod
    def prettyjson(txt:str) -> str:
        """
        Pretty-print a JSON-compatible object as a formatted string.
        Args:
            txt (str): JSON-compatible object.
        Returns:
            str: Indented JSON string.
        """
        return str(json.dumps(txt,indent=4, ensure_ascii=False))
    
    @staticmethod
    def to_literal(value):
        """
        Convert multiline strings into LiteralString for YAML output.
        Single-line strings and non-string values are returned unchanged.
        """
        if isinstance(value,str) and "\n" in value:
            return LiteralString(value)
        return value
    
    @staticmethod
    def deep_literal_transform(data):
        '''
        Recursively convert ALL multiline strings inside dict/list
        into LiteralString so block formatting is preserved
        '''
        if isinstance(data, dict):
            return {k: Helper.deep_literal_transform(v) for k,v in data.items()}
        if isinstance(data, list):
            return [Helper.deep_literal_transform(i) for i in data]
        # Convert only multiline string
        return Helper.to_literal(data)

In [71]:
from google import genai
import json
import os
import yaml

class PromptBuilder(Helper):
    """
    Builds a structured evaluation prompt for a specific resume section.
    Uses YAML-driven configuration to assemble role instructions,
    evaluation criteria, scoring scale, expected content, and
    the candidate resume into a single LLM-ready prompt.
    """

    def __init__(self, section, criteria, targetrole, cvresume, include_fewshot: bool = True, output_lang = "en"):
        """
        Initialize the prompt builder for a resume section.
        Args:
            section (str): Resume section to evaluate (e.g. "Education").
            criteria (list): Evaluation criteria for the section.
            targetrole (str): Target role used for role relevance.
            cvresume (str): Resume content to be evaluated.
            include_fewshot (bool): Whether to include example scores.
            output_lang (str): Output language code (e.g. "en", "th").
        """
        self.section        = section
        self.criteria       = criteria[::-1]
        self.cvresume       = cvresume
        self.targetrole     = targetrole
        self.include_fewshot= include_fewshot
        self.output_lang    = output_lang
        
        self.config = self.load_yaml(r"C:\Users\TunKedsaro\Desktop\CVResume\src\config\prompt.yaml")
        self.criteria_cfg = self.config.get("criteria", {})

    def build_response_template(self):
        """
        Build an empty JSON response template for the LLM.
        Returns:
            dict: Response skeleton with section name and criteria scores.
        """
        return {
            "section": self.section,
            "scores": {
                c: {"score": 0, "feedback": ""} for c in self.criteria
            }
        }
    
    def _build_criteria_block(self) -> str:
        """
        Construct the criteria description block for the prompt.
        Includes few-shot score examples if enabled and available
        in the prompt configuration.
        Returns:
            str: Formatted criteria block text.
        """
        blocks = []
        for crit in self.criteria:
            block = f"- {crit}\n"
            if self.include_fewshot and crit in self.criteria_cfg:
                few_cfg = self.criteria_cfg[crit]
                for score in [5, 3, 1]:
                    key = f"score{score}"
                    if key in few_cfg:
                        text = few_cfg[key].strip()
                        block += f"    score {score}: {text}\n"
            blocks.append(block)
        return "".join(blocks)
    
    def build(self):
        """
        Assemble the full evaluation prompt.
        Combines role instructions, evaluation objectives, section context,
        criteria definitions, scoring scale, expected output format,
        and resume content into a single prompt string.
        Returns:
            str: Final prompt ready to be sent to the LLM.
        """
        config_role      = self.config['role']['role1']
        config_objective = self.config['objective']['objective1']
        config_section   = self.config['section']['section1']
        config_expected  = self.config['expected_content'][self.section]
        config_scale     = self.config['scale']['score1']
        criteria_block   = self._build_criteria_block()
        config_lang      = self.config['Language_output_style'][self.output_lang]

        prompt_role      = f"Role :\n{config_role}\n\n"
        prompt_objective = f"Objectvie :\n{config_objective}\n"
        promnt_lang      = f"Output Language Instruction::\n{config_lang}\n"
        prompt_section   = f"Section :\n{config_section}\n\n"
        prompt_expected  = f"Expected :\n{config_expected}\n"
        prompt_criteria  = f"Criteria :\n{criteria_block}\n"
        prompt_scale     = f"Scale :\n{config_scale}\n"
        prompt_output    = f"Otput :\n{json.dumps(self.build_response_template(), indent=2)}\n\n"
        prompt_cvresume  = f"CV/Resume: \n{self.cvresume}\n"
        prompt = (
            prompt_role + prompt_objective + prompt_section + promnt_lang
            + prompt_expected + prompt_criteria + prompt_scale
            + prompt_output + prompt_cvresume 
        )
        # print(f"prompt -> \n{prompt}")
        prompt = prompt.replace("<section_name>", self.section)
        prompt = prompt.replace("<targetrole>", self.targetrole)
        return prompt

In [72]:
def evaluate_resume(payload):
    resume_json = payload.resume_json
    targetrole  = payload.target_role
    output_lang = payload.output_lang 

    p1 = PromptBuilder(
        section     = "Profile",
        criteria    = ["Completeness", "ContentQuality"],
        targetrole  = targetrole,
        cvresume    = resume_json,
        output_lang = output_lang 
    )
    prompt1 = p1.build()
    
    p2 = PromptBuilder( 
        section     = "Summary", 
        criteria    = ["Completeness", "ContentQuality","Grammar","Length","RoleRelevance"],
        targetrole  = targetrole,
        cvresume    = resume_json,
        output_lang = output_lang 
    )
    prompt2 = p2.build()

    # op1,raw1 = caller.call(prompt1)
    # op2,raw2 = caller.call(prompt2)

    # s1 = agg.aggregate(op1)
    # s2 = agg.aggregate(op2)

    # x = GlobalAggregator(SectionScoreAggregator_output = [s1,s2])

    # output = x.fn0()


    # return {
    #     "response": output,
    #     "response_time" : f"{usage_time:.5f} s",
    #     "estimated_cost_thd": f"{(cost1['total_cost']+cost2['total_cost']+cost3['total_cost']+cost4['total_cost']+cost5['total_cost']+cost6['total_cost'])*usd2bath:.5f} ฿"
    # }


In [ ]:


import asyncio

from pydantic import BaseModel, Field, ValidationError
from typing import Dict

class CriterionScore(BaseModel):
    """
    Represents a single evaluation score for one criterion.
    Attributes:
        score (int): Integer score between 0 and 5.
        feedback (str): Textual feedback explaining the score.
    """
    score: int = Field(ge=0, le=5)
    feedback: str

class SectionEvaluation(BaseModel):
    """
    Validated LLM evaluation output for a resume section.
    Attributes:
        section (str): Name of the resume section.
        scores (Dict[str, CriterionScore]): Mapping of criteria to scores.
    """
    section: str
    scores: Dict[str, CriterionScore]

class LlmCaller(Helper):
    def __init__(self):
        """
        Handles interaction with the LLM for section-level resume evaluation.
        Responsibilities:
        - Send prompts to the LLM
        - Parse and validate structured JSON responses
        - Retry with a repair prompt if validation fails
        - Return validated output along with raw LLM response
        """
        self.model_cfg = self.load_yaml(r"C:\Users\TunKedsaro\Desktop\CVResume\src\config\model.yaml")
        self.client    = genai.Client(api_key='AIzaSyARN...L2b-3dHAWb5urRk')
        self.model     = self.model_cfg["model"]["generation_model"]
        self.usd2bath  = Helper.load_yaml(r"C:\Users\TunKedsaro\Desktop\CVResume\src\config\global.yaml")['currency']['USD_to_THB']
        self.log_digit = Helper.load_yaml(r"C:\Users\TunKedsaro\Desktop\CVResume\src\config\global.yaml")['logging']['logging_round_digit']

    def _parse(self, resp):
        """
        Parse raw LLM response text into a JSON object.
        Removes markdown code fences and converts the result to a dictionary.
        Args:
            resp: Raw response object from the LLM client.
        Returns:
            dict: Parsed JSON output.
        """
        text = resp.text.strip()
        text = re.sub(r"^```json|```$", "", text).strip()
        return json.loads(text)
    
    def _call_raw(self, prompt: str):
        """
        Send a prompt to the LLM without validation.
        Args:
            prompt (str): Prompt text to send.
        Returns:
            tuple:
                - Parsed JSON output (dict)
                - Raw LLM response object
        """
        resp = self.client.models.generate_content(
            model=self.model,
            contents=prompt
        )
        parsed = self._parse(resp)
        return parsed, resp
    
    def _validate(self, raw_output:dict)->SectionEvaluation:
        """
        Validate LLM output against the expected schema.
        Args:
            raw_output (dict): Parsed LLM output.
        Returns:
            SectionEvaluation: Validated evaluation object.
        Raises:
            ValidationError: If the output does not match the schema.
        """
        return SectionEvaluation.model_validate(raw_output)
    
    def _repair_prompt(self, error_msg: str) -> str:
        """
        Generate a corrective prompt when LLM output validation fails.
        Instructs the LLM to strictly follow the expected JSON schema.
        Args:
            error_msg (str): Validation error message.
        Returns:
            str: Repair prompt to prepend to the original prompt.
        """
        return f"""
                Your previous response was INVALID.
                Validation error:
                {error_msg}
                STRICT RULES:
                - Return JSON only
                - No markdown
                - No explanation
                - Follow schema exactly
                - Section name must start with a capital letters (e.g. "Education")
                Expected format:
                {{
                    "section": "<section_name>",
                    "scores": {{
                        "<criterion>": {{ "score": 0-5, "feedback": "string" }}
                    }}
                }}
        """
    
    def call(self,prompt:str, max_retry:int = 3):
        """
        Call the LLM with validation and automatic retry.
        Attempts to validate the LLM response. If validation fails,
        retries using a repair prompt up to the specified limit.
        Args:
            prompt (str): Prompt text to send.
            max_retry (int): Maximum number of retry attempts.
        Returns:
            tuple:
                - Validated evaluation result (dict)
                - Raw LLM response object
        """
        last_error = None
        repair_prompt = "\n"
        for attemp in range(max_retry):
            final_prompt = repair_prompt + prompt
            # print(f"final_prompt attemp : {attemp} -> \n {final_prompt}")
            output, raw = self._call_raw(final_prompt)
            try:
                validated = self._validate(output)
                # print('Status : 1')
                return validated.model_dump(),raw
            except ValidationError as e:
                last_error = str(e)
                repair_prompt = self._repair_prompt(last_error)
                # print('Status : 0')
                print("Output error recall again ...")
            finally:
                print('='*100)
        return {
            "section":"UNKNOW",
            "scores":{}
        },raw

    async def call_async(self, prompt: str):
        """
        Asynchronous wrapper for `call`.
        Executes the blocking LLM call in a background thread.
        Args:
            prompt (str): Prompt text to send.
        Returns:
            tuple:
                - Validated evaluation result (dict)
                - Raw LLM response object
        """
        return await asyncio.to_thread(self.call, prompt)


In [ ]:
from datetime import datetime,timezone,timedelta
import copy

class SectionScoreAggregator(Helper):
    """
    Aggregates raw LLM criterion scores for a single resume section
    into weighted scores and a section-level total score (0-5).
    Configuration:
        - src/config/weight.yaml
    Expected LLM Output Format (input):
        {
            "section": "<section_name>",
            "scores": {
                "<criterion>": {
                    "score": int,        # 0-5
                    "feedback": str
                }
            }
        }

    Output Format:
        {
            "section": "<section_name>",
            "total_score": float,
            "scores": {
                "<criterion>": {
                    "score": float,      # weighted score
                    "feedback": str
                }
            }
        }
    """
    def __init__(self):
        self.config          = self.load_yaml(r"C:\Users\TunKedsaro\Desktop\CVResume\src\config\weight.yaml")         # config
    def aggregate(self,llm_output:dict):
        self.llm_output      = llm_output             # op
        self.section         = llm_output["section"]  # Get section
        self.section_weights = self.config["weights"][self.section]  # config["weights"][section_key][criteria]
        ddict = {}
        total = 0.0
        # Protect multiple mutation when we run more than one time
        scores_copy = copy.deepcopy(self.llm_output["scores"])
        for criteria, body in scores_copy.items():
            raw = body["score"]
            w   = self.section_weights[criteria]
            weighted = raw / 5 * w
            body["score"] = weighted
            ddict[criteria] = body
            total = total + weighted
        return {
            "section": self.section,
            "total_score":total,
            "scores":ddict
        }
    
    
class GlobalAggregator(Helper):
    """
    Aggregates section-level evaluation results into a final resume score.
    Combines weighted section scores, detailed per-section breakdowns,
    and evaluation metadata (model, configuration versions, timestamp).
    Inputs:
        SectionScoreAggregator_output (list):
            Section-level aggregation results.
    Configurations:
        - model.yaml   : LLM model information
        - weight.yaml  : Section weights and version
        - prompt.yaml  : Prompt configuration version
    Input : 
    s1 = {
    'section': 'Experience',
    'total_score': 78.0,
    'scores': {
        'RoleRelevance':  {'score': 30.0, 'feedback': 'xxx'},
        'Length':         {'score': 8.0, 'feedback': 'yyy'},
        'Grammar':        {'score': 10.0, 'feedback': 'zzz'},
        'ContentQuality': {'score': 24.0, 'feedback': 'aaa'},
        'Completeness':   {'score': 6.0, 'feedback': 'bbb'}
            }
        }
    s2 = {
        'section': 'Profile',
        'total_score': 90.0,
        'scores': {
            'RoleRelevance':  {'score': 20.0, 'feedback': 'xxx'},
            'Length':         {'score': 10.0, 'feedback': 'yyy'},
            'Grammar':        {'score': 25.0, 'feedback': 'zzz'},
            'ContentQuality': {'score': 22.0, 'feedback': 'aaa'},
            'Completeness':   {'score': 13.0, 'feedback': 'bbb'}
                }
            }

    Output :
    Ss = {
        "conclution":{
            "final_resume_score": 82.4,
            "section_contribution":{
                "Profile":{
                    "section_total":78.0,
                    "section_weight":40,
                    "contribution":31.2
                },
                "Experience":{
                    "section_total":90.0,
                    "section_weight":10,
                    "contribution":9.0
                }
            }
        },
        "section_details":{
            "Profile":{
                "total_score":78.0,
                "scores":{
                        'RoleRelevance':  {'score': 30.0, 'feedback': 'xxx'},
                        'Length':         {'score': 8.0, 'feedback': 'yyy'},
                        'Grammar':        {'score': 10.0, 'feedback': 'zzz'},
                        'ContentQuality': {'score': 24.0, 'feedback': 'aaa'},
                        'Completeness':   {'score': 6.0, 'feedback': 'bbb'}
                }
            },
            "Experience":{
                "total_score":90.0,
                "scores":{
                        'RoleRelevance':  {'score': 20.0, 'feedback': 'xxx'},
                        'Length':         {'score': 10.0, 'feedback': 'yyy'},
                        'Grammar':        {'score': 25.0, 'feedback': 'zzz'},
                        'ContentQuality': {'score': 22.0, 'feedback': 'aaa'},
                        'Completeness':   {'score': 13.0, 'feedback': 'bbb'}
                }      
            }
        },
        "metadata":{
            "model_name":"gemini-2.5-flash",
            "timestamp": "2025-12-03T11:45:00+07:00",
            # "processing_time_ms":1234,
            # "input_tokens":1234,
            # "output_tokens":435,
            # "total_cost_usd":0.001234,
            "weights_version":"weights_v1",
            "prompt_version":"prompt_v1"
        }
    }                                                                                       

    """
    def __init__(self,SectionScoreAggregator_output:list):
        self.section_outputs = SectionScoreAggregator_output
        self.timestamp       = str(datetime.now(tz=(timezone(timedelta(hours=7)))))
        self.model_config    = Helper.load_yaml(r"C:\Users\TunKedsaro\Desktop\CVResume\src\config\model.yaml")     # should include model name
        self.weight_config   = Helper.load_yaml(r"C:\Users\TunKedsaro\Desktop\CVResume\src\config\weight.yaml")    # includes weights + version
        self.prompt_config   = Helper.load_yaml(r"C:\Users\TunKedsaro\Desktop\CVResume\src\config\prompt.yaml")  
        self.model_cfg       = self.load_yaml(r"C:\Users\TunKedsaro\Desktop\CVResume\src\config\model.yaml")
        self.client    = genai.Client(api_key='AIzaSyARNu...gL2b-3dHAWb5urRk')
        self.model     = self.model_cfg["model"]["generation_model"]
    # includes prompt version
    def fn1(self,globalfeedback):
        """
        Calculate the final resume score using section-level weights.
        Applies section weights to each section's total score and
        computes the overall composite resume score.
        Returns:
            dict:
                Final resume score and per-section contribution details.
        """
        weights = self.weight_config["weights"]
        contribution = {}
        total = 0.0
        for section_data in self.section_outputs:
            section_name    = section_data["section"]
            total_score     = section_data["total_score"]
            section_weight  = weights[section_name]["section_weight"]
            section_contrib = total_score * section_weight
            contribution[section_name] = {
                "section_total": total_score,
                "section_weight": section_weight,
                "contribution": Helper.fop(section_contrib)
            }
            total = total + section_contrib
        return {
            "final_resume_score":Helper.fop(total),
            "section_contribution":contribution,
            "globalfeedback":globalfeedback
        }
    def fn2(self):
        print("fn2")
        details = {}
        for section_data in self.section_outputs:
            details[section_data["section"]] = {
                'total_score':section_data['total_score'],
                'scores':section_data['scores']
            }
        # print(f"detail -> {details}")
        prompt = f'''
            You have to give me Global feedback from this cvresume
            this is session evaluate for every part of resume you have to read
            for each comment then give me overall feedback
            {details}
        '''
        # print(f"prompt -> {prompt}")
        resp = self.client.models.generate_content(
            model=self.model,
            contents=prompt
        )
        # print(resp.text)
        return details,resp.text
    
    def fn3(self):
        """
        Generate metadata describing the evaluation context.
        Includes model information, configuration versions,
        and the evaluation timestamp.
        Returns:
            dict:
                Evaluation metadata.
        """
        return {
            "model_name": self.model_config['model']['generation_model'],
            "timestamp": self.timestamp,
            "weights_version": self.weight_config.get("version", "unknown"),
            "prompt_version": self.prompt_config.get("version", "unknown")
        }
    def fn0(self):
        """
        Assemble the final evaluation response.
        Combines the final resume score, detailed section results,
        and evaluation metadata into a single response payload.
        Returns:
            dict:
                Final aggregated resume evaluation result.
        """
        detail_part,globalfeedback     = self.fn2()
        conclution_part = self.fn1(globalfeedback)
        metadata_part   = self.fn3()
        # return {
        #     "conclution":conclution_part,
        #     "section_detail":detail_part,
        #     "metadata":metadata_part
        # }
        return {
            "conclution":conclution_part,
            "section_detail":detail_part,
            "metadata":metadata_part
        }

<hr>

In [104]:
resume_json = Helper.load_json(r"C:\Users\TunKedsaro\Desktop\CVResume\src\mock\resume3.json")
targetrole = "data scientist"
output_lang = "en"

In [119]:
caller    = LlmCaller()
agg       = SectionScoreAggregator()

In [120]:
p1 = PromptBuilder(
    section     = "Profile",
    criteria    = ["Completeness", "ContentQuality"],
    targetrole  = targetrole,
    cvresume    = resume_json,
    output_lang = output_lang 
)
prompt1 = p1.build()
p2 = PromptBuilder( 
    section     = "Summary", 
    criteria    = ["Completeness", "ContentQuality","Grammar","Length","RoleRelevance"],
    targetrole  = targetrole,
    cvresume    = resume_json,
    output_lang = output_lang 
)
prompt2 = p2.build()
op1,raw1 = caller.call(prompt1)
op2,raw2 = caller.call(prompt2)

In [136]:
s1 = agg.aggregate(op1)
s2 = agg.aggregate(op2)
x = GlobalAggregator(SectionScoreAggregator_output = [s1,s2])

In [137]:
output = x.fn0()

fn2


In [145]:
print(output['conclution']['globalfeedback'])

This resume presents a strong and professional foundation, particularly in its clarity, conciseness, and strong alignment with a Data Scientist role.

Here's an overall evaluation based on the provided feedback for the 'Profile' and 'Summary' sections:

**Overall Strengths:**

1.  **Clear Professional Identity & Role Relevance:** Both the Profile and Summary effectively position the candidate as a Data Scientist, highlighting their passion, background, and the value they bring through data analysis and insights. The Summary, in particular, achieves excellent role relevance.
2.  **Exceptional Conciseness and Grammar:** The Summary is lauded for its perfect length and impeccable grammar, contributing to a highly professional and easy-to-read first impression. This indicates strong communication skills.
3.  **Comprehensive Coverage:** Within their respective constraints, both sections are complete. The Profile covers essential identity and direction, while the Summary effectively touches 

In [141]:
print(str(json.dumps(output,indent=4, ensure_ascii=False)))

{
    "conclution": {
        "final_resume_score": 6.2,
        "section_contribution": {
            "Profile": {
                "section_total": 16.0,
                "section_weight": 0.1,
                "contribution": 1.6
            },
            "Summary": {
                "section_total": 46.0,
                "section_weight": 0.1,
                "contribution": 4.6
            }
        },
        "globalfeedback": "This resume presents a strong and professional foundation, particularly in its clarity, conciseness, and strong alignment with a Data Scientist role.\n\nHere's an overall evaluation based on the provided feedback for the 'Profile' and 'Summary' sections:\n\n**Overall Strengths:**\n\n1.  **Clear Professional Identity & Role Relevance:** Both the Profile and Summary effectively position the candidate as a Data Scientist, highlighting their passion, background, and the value they bring through data analysis and insights. The Summary, in particular, achieves exc

<hr>

In [173]:
class test(LlmCaller):
    def __init__(self):
        super().__init__()

In [176]:
x = test()

In [180]:
x.model_cfg
x.client

In [187]:
output, raw = x._call_raw("Hey how are you doing (return json format)")

In [188]:
output

{'greeting': 'Hello there!',
 'status': "I'm doing great, thank you for asking.",
 'mood': 'Positive and ready to assist.',
 'details': "As an AI, I don't experience feelings in the human sense, but my systems are running perfectly."}